- https://www.philipzucker.com/egraph2024_talk_done/
- https://www.philipzucker.com/linear_grobner_egraph/
- https://www.philipzucker.com/coegraph/
- https://www.philipzucker.com/string_knuth/
- https://www.philipzucker.com/smart_constructor_aegraph/
- https://www.philipzucker.com/multiset_rw/
- https://www.philipzucker.com/bottom_up/ normalizing containers


Enodes ~ syntax
eids ~ semantics

When you ask z3 for a model, it returns numerical semantic values.

Equality saturation starts as pure syntactic equality and gradually moves over closer to semantic equality (which is often never reaches).

There are two main approaches I've been trying (or that I'm aware of):

- Containers as smarter enodes
- structured / semantic eids

The enode structure I used is a function symbol + an ordered list of arguments. This is already kind of smart compared to a function symbol + a binary tuple of args. This tuple form can be interesting if you want to talk about partial application.


Destructive rewriting as a theory.
If you have a good rewrite theory over a system (terminating and confluent), you can already effectively quotient the syntax trees by that notion of equivalence be greedily applying the rules.

```
add(succ(X), Y) -> succ(add(X,Y))
add(Z, Y) -> Y
```


Combination problems
https://www.sciencedirect.com/science/article/pii/S0890540101931189 Deciding the Word Problem in the Union of Equational Theories

Canonization for disjoint unions of theories Sava Krstic  Sylvain Conchon

Disjoint signatures
Termination is not modular
Confluence is modular
Normalizing is modular

Ground rewrite rules / equalities are quite special.

The following no go counterexample crushes most hopes of generic good results.

Associativity + ground term equations = string equality and is undecidable.

You can build a normalizing rewrite rule system for associativity by associating to the right.
`(x . y) . z -> x . (y . z)`

Ok but actually string rewrite systems are rules of form (yadayda . X) -> (something . X). They are not ground. Yeah but associatvity can make them non ground
A ground _term_ equation corresponds more directly to the string rewrite that only applies at the end of the string. 

For disjoint signatures, there doesn't seem like their should be a problem. Why were eids integers in the first place? Who cares? The pieces of the other signature completely block rules from applying.


Pick a ground total termination ordering compatible with the rewrite relation.
COnsider the particular stratgey: orient the built in rules, All confluence pairs are now redundant by assumption.
Start orienting the ground equations. simplify reduces by the built ins. This is "putting into the egraph".
Any symbol not part of built ins can have a generated guy.




Ok, so we want non-disjoint signatures.


What terms are "in" the egraph:
1. A generative point of view (top down production of terms). Given an eid, we can build. The view is related to top down ematching
2. An acceptor point of view. If we insert of term bottom up into the egraph, does it change it or leave it the same? This view is related to bottom up ematching.

We can consider maybe adding new function symbols over 


Can we put eids and enodes on the same footing? If I had 3 theories I wanted to bake in. Does baked in go into eids? Or 2 notions of eids?

https://egraphs.zulipchat.com/#narrow/channel/328972-general/topic/Linear.20and.20Polynomial.20Equations/near/477290392


https://egraphs.zulipchat.com/#narrow/channel/424128-egg.2Fgeneral/topic/Translating.20between.20languages.20in.20an.20egraph/near/499111241 translation between mutiple languages slows stuff down.
Related to refinement egraph ideas?


|           |   UF    | Theory |
| ---       |  ---    |        |
| Sorts     | 
| Enodes    |
| Eclasses  |
| Functions |
| Rules     |   


Egraph modulo theories:
What is a theory?

https://pldi25.sigplan.org/home/egraphs-2025


In [2]:
class SortRef():
    def __init__(self, name):
        self.name = name
        self.uf = []
        # self.utree # to maintain eclasses
        # self.reason # to maintain provencnance / proof
    def find(self, i): # canonize
        while self.uf[i] != i:
            i = self.uf[i]
        return i
    def union(self, i, j):
        i = self.find(i)
        j = self.find(j)
        if i != j:
            self.uf[i] = j
    def make(self): # Const?
        self.uf.append(len(self.uf))
        return len(self.uf)-1
        

class FuncDeclRef(): # MergeDict
    def __init__(self, name, *sorts):
        self.data = {}
        self.sorts = sorts
    def __getitem__(self, args):
        assert len(args) == len(self.sorts) - 1
        return self.data[args]
    def __call__(self, *args):
        assert len(args) == len(self.sorts) - 1
        res = self.data.get(args)
        if res is None:
            res = self.default()
            self.data[args] = res
            return res
        else:
            return res
    def __setitem__(self, key, value):
        assert len(key) == len(self.sorts) - 1
        res = self.data.get(key)
        if res is None:
            self.data[key] = value
        else:
            self.data[key] = self.merge(res, value)
    def default(self):
        return self.sorts[-1].make()
    def merge(self,i,j):
        return self.sorts[-1].union(i,j)
    def rebuild(self):
        fnew = FuncDeclRef(self.name, *self.sorts)
        for k,v in self.data.items():
            newk = tuple(sort.canon(k) for k,sort in zip(k,self.sorts[:-1]))
            fnew[newk] = self.sorts[-1].canon(v)
        return fnew
 
Int = SortRef("Int")
add = FuncDeclRef("add", Int, Int, Int)
x = FuncDeclRef("x", Int)()
add(x,x)


1

In [7]:
class MergeDict(dict):
    def __setitem__(self, key, value):
        res = self.get(key)
        if res is None:
            super().__setitem__(key, value)
        else:
            super().__setitem__(key, self.merge(res, value))
    def merge(self, i, j):
        raise NotImplementedError
    
class MinDict(MergeDict):
    def merge(self, i, j):
        return min(i, j)

class MaxDict(MergeDict):
    def merge(self, i, j):
        return max(i, j)

d = MinDict()
d[1] = 2
d[1] = 3
assert d[1] == 2
d[1] = 1
assert d[1] == 1

In [ ]:
class MSFuncDeclRef(FuncDeclRef):
    # multiset funcdecl ref. 
    # Putting the "ACness" into the functions
    # Containers ~ enodes
    def rebuild(self):
        fnew = MSFuncDeclRef(self.name, *self.sorts)
        for k,v in self.data.items():
            fnew[k] = v
        return fnew


In [ ]:
class ACSort():
    # putting the "ACness" in the sort.
    # structured eids which are multisets
    # containers ~ eids
    def __init__(self, name, inner):
        self.name = name
        self.inner = inner
    def rebuild(self, xs : tuple): # rebuild / re-canonize
        return tuple(sorted([self.inner.rebuild(x) for x in xs]))

class ACISort():
    # ACI ~ sets
    def __init__(self, name, inner):
        self.name = name
        self.inner = inner
    def rebuild(self, xs : tuple): # rebuild / re-canonize
        

A notion of "kinds". Sorts may have different structured ids. The description or data of thse may lie in the Kind data strucctures.

In [ ]:
class UninterpretedKind():
    def __init__(self, name):
        self.uf = []
    def make(self):
        self.uf.append(len(self.uf))
        return len(self.uf)-1
    def union(self, i, j):
        return i
    def canon(self, i):
        return i
    def rebuild(self, i):
        return self.find(i)


class SortRef():
    def __init__(self, name, kind=None):
        self.name = name
        if kind is None:
            kind = UninterpretedKind()
        else:
            self.kind = kind
    

We could easily have multiple notions of equality.
They don't necessarily have to have much to do with each other. But we can't normalize across them except at the commonality.
There is a truly baked in equality that can be destructive.

We could only canonize when they agree they canonize. Or maintain a fixpoint of the canonization.


In [ ]:
class AtomicEq():
    def __init__(self, sort):
        self.sort = sort
        self.uf = []
    def canon(self, i):
        pass
    def rebuild():

class LinearEq():
    def __init__(self, sort):
        self.sort = sort
        self.uf = []
    def canon(self, i):
        pass
    def rebuild():
        pass # do gauss elim?

class RecursiveEq(): # FixEq
    pass # todo


class GroupActEq():
    pass # Can have group actions on the side of the UF

class StringEq(): ...

class FixEq(): ...

class UnionEq():
    def __init__():
        pass

We are quotienting by different equalities.
Having them play nice with each other is a theory combination problem.

Partitions ~ equaiovlanece relations.
Finest partition or corasest partition that contains the equivalence relations.
Equivalence relations form a lattice.
https://en.wikipedia.org/wiki/Equivalence_relation

We can bounce around two equivalence relations is we have a notion of well founded ordering we are following and get the canonizer for the combo (the coarser partition with more equalities).

I don't know how we'd get a canonizer for the intersection.

In [ ]:
class QuotientSort():
    def __init__(self, sort, eq):
        self.sort = sort
        self.eq = eq


In [ ]:
class ValueKind():
    # literals like literal integers
    def rebuild(self, i):
        return i
    
# This makes sense for Int where we have abstract Int expressions, but also sometimes literal int values. "Lifting" is somewhat awkward.
# Perhaps related to my notion of observation table for coegraphs. A ground value is a very powerful observation.


In [ ]:
class UnionKind(): # union or disjoint union / tagged union?
    def __init__(self, *kinds):
        self.kinds = kinds
    def rebuild(self, i):
        tag, inner = i
        return (tag, self.kinds[tag].rebuild(inner))
    def make(self, tag): # Hmm. Make takes stuff now...
        return (tag, self.kinds[tag].make())

class IntersectionKind():
    

From the ground rewrite system perspective, what are we doing?

A specialized notion of overlap
a compositional form of term ordering. Making a term ordering from a term ordering of the contained elements.
a notion of pattern matching and replacement / removal



TermKind

In [ ]:
def ACDict():
    

In [ ]:

class ExprRef():
    def __init__(self, id_, sort):
        self.sort = sort
        self.id_ = id_
    def __eq__(self, other):
        assert self.sort is other.sort
        self.sort.union(self, other)
    def eq(self, other):
        assert self.sort is other.sort
        return self.sort.find(self.id_) == self.sort.find(other.id_)

: